In [1]:
cd D:\My_Projects\Sponsorlytix_Assignment

D:\My_Projects\Sponsorlytix_Assignment


d:\My_Projects\Sponsorlytix_Assignment\venv3.9\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Frame extration for video

In [2]:
import os
import cv2
import yt_dlp

def extract_frames(video_path, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_folder, f"frame_{frame_count:05d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames to {output_folder}")

video_path="Data/v2.mp4"
output_image_path="Data/images_2"

if __name__ == "__main__":
    extract_frames(video_path, output_image_path)


Extracted 4900 frames to Data/images_2


### Data augmentation

In [19]:
import os
import cv2
import numpy as np
import shutil
import albumentations as A
from tqdm import tqdm

# Paths
INPUT_IMAGE_DIR = r"D:\My_Projects\Sponsorlytix_Assignment\Data\Dataset\test\images"
INPUT_LABEL_DIR = r"D:\My_Projects\Sponsorlytix_Assignment\Data\Dataset\test\labels"
OUTPUT_IMAGE_DIR = r"D:\My_Projects\Sponsorlytix_Assignment\yolov7\Dataset\test\images"
OUTPUT_LABEL_DIR = r"D:\My_Projects\Sponsorlytix_Assignment\yolov7\Dataset\test\labels"

# Create output directories
os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)
os.makedirs(OUTPUT_LABEL_DIR, exist_ok=True)

# Define augmentations (Only shape-preserving)
augmentations = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        #A.RandomRotate90(p=0.5),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.7),
        A.Affine(scale=(0.8, 1.2), translate_percent=(0.1, 0.2), rotate=(-30, 30), shear=(-10, 10), p=0.7),

    ],
    bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])
)

# Helper function to read annotations
def read_yolo_annotation(label_path):
    with open(label_path, "r") as file:
        bboxes = []
        class_labels = []
        for line in file:
            parts = line.strip().split()
            class_labels.append(int(parts[0]))
            bbox = list(map(float, parts[1:]))
            bboxes.append(bbox)
        return bboxes, class_labels

# Helper function to write annotations
def write_yolo_annotation(output_path, bboxes, class_labels):
    with open(output_path, "w") as file:
        for cls, bbox in zip(class_labels, bboxes):
            bbox_line = f"{int(cls)} " + " ".join(map(str, bbox))
            file.write(bbox_line + "\n")

# Process dataset
for image_file in tqdm(os.listdir(INPUT_IMAGE_DIR), desc="Processing Images"):
    if image_file.endswith((".jpg", ".png", ".jpeg")):
        # Load image and label
        image_path = os.path.join(INPUT_IMAGE_DIR, image_file)
        
        # Handling label file variations
        label_filename = image_file.rsplit(".", 1)[0]  # Remove extension
        label_path = os.path.join(INPUT_LABEL_DIR, f"{label_filename}.txt")

        # Try alternative label name format (e.g., frame_00243.xml.txt)
        if not os.path.exists(label_path):
            alternative_label_path = os.path.join(INPUT_LABEL_DIR, f"{label_filename}.xml.txt")
            if os.path.exists(alternative_label_path):
                label_path = alternative_label_path
            else:
                print(f"Skipping {image_file}: No corresponding label file found.")
                continue

        # Read image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Skipping {image_file}: Failed to load image.")
            continue
        
        img_height, img_width = image.shape[:2]

        # Ensure image shape remains constant
        if (img_width, img_height) != (1280, 720):
            print(f"Skipping {image_file}: Image shape changed.")
            continue

        # Read bounding boxes
        bboxes, class_labels = read_yolo_annotation(label_path)

        # Copy original image & label to output directory
        shutil.copy(image_path, os.path.join(OUTPUT_IMAGE_DIR, image_file))
        shutil.copy(label_path, os.path.join(OUTPUT_LABEL_DIR, f"{label_filename}.txt"))

        if len(bboxes) == 0:
            continue  # Skip if no bounding boxes

        # Apply augmentation
        for i in range(5):  # Generate 5 augmented versions per image
            augmented = augmentations(image=image, bboxes=bboxes, class_labels=class_labels)

            augmented_image = augmented["image"]
            augmented_bboxes = augmented["bboxes"]
            augmented_class_labels = augmented["class_labels"]

            # Convert image back to BGR before saving
            augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)

            # Save augmented image
            aug_image_path = os.path.join(OUTPUT_IMAGE_DIR, f"{image_file[:-4]}_aug{i}.jpg")
            cv2.imwrite(aug_image_path, augmented_image)

            # Save augmented labels
            aug_label_path = os.path.join(OUTPUT_LABEL_DIR, f"{image_file[:-4]}_aug{i}.txt")
            write_yolo_annotation(aug_label_path, augmented_bboxes, augmented_class_labels)



Processing Images:   0%|          | 0/12 [00:00<?, ?it/s]

Processing Images: 100%|██████████| 12/12 [00:01<00:00,  8.24it/s]


### Ploting

In [20]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Paths
IMAGE_DIR = r"D:\My_Projects\Sponsorlytix_Assignment\yolov7\Dataset\test\images"
LABEL_DIR = r"D:\My_Projects\Sponsorlytix_Assignment\yolov7\Dataset\test\labels"
OUTPUT_DIR = r"D:\My_Projects\Sponsorlytix_Assignment\yolov7\Dataset\test\visualized"

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Class names (modify according to your dataset)
CLASS_NAMES = {
    0: "Jab",
    1: "Punching bag",
    2: "Hook",
    3: "Uppercut"
}

# Colors for different classes
COLORS = [
    (0, 255, 0),  # Green
    (255, 0, 0),  # Blue
    (0, 0, 255),  # Red
    (255, 255, 0)  # Cyan
]

# Helper function to read YOLO annotations
def read_yolo_annotation(label_path, img_width, img_height):
    with open(label_path, "r") as file:
        bboxes = []
        class_labels = []
        for line in file:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])
            
            # Convert YOLO format (normalized) to pixel coordinates
            x_min = int((x_center - width / 2) * img_width)
            y_min = int((y_center - height / 2) * img_height)
            x_max = int((x_center + width / 2) * img_width)
            y_max = int((y_center + height / 2) * img_height)
            
            bboxes.append((x_min, y_min, x_max, y_max))
            class_labels.append(class_id)
    
    return bboxes, class_labels

# Loop through images
for image_file in tqdm(os.listdir(IMAGE_DIR), desc="Visualizing Annotations"):
    if image_file.endswith((".jpg", ".png", ".jpeg")):
        image_path = os.path.join(IMAGE_DIR, image_file)
        label_path = os.path.join(LABEL_DIR, image_file.rsplit(".", 1)[0] + ".txt")

        if not os.path.exists(label_path):
            print(f"Skipping {image_file}: No corresponding label file found.")
            continue

        # Load image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Skipping {image_file}: Could not load image.")
            continue

        img_height, img_width = image.shape[:2]

        # Read bounding boxes
        bboxes, class_labels = read_yolo_annotation(label_path, img_width, img_height)

        # Draw bounding boxes on the image
        for bbox, class_id in zip(bboxes, class_labels):
            x_min, y_min, x_max, y_max = bbox
            color = COLORS[class_id % len(COLORS)]  # Assign color based on class

            # Draw rectangle
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)

            # Put class label
            label = CLASS_NAMES.get(class_id, f"Class {class_id}")
            cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Save the visualized image
        output_path = os.path.join(OUTPUT_DIR, image_file)
        cv2.imwrite(output_path, image)

print("✅ Annotation visualization completed. Check the 'visualized' folder.")


Visualizing Annotations: 100%|██████████| 72/72 [00:01<00:00, 39.61it/s]

✅ Annotation visualization completed. Check the 'visualized' folder.
